In [51]:
from typing import Tuple, Any

In [71]:
task = """020D64AEE52E55B4C017938FBBAC2D6002A53D21F9E90C18023600B80021D0862DC1700043232C2284D3B0105007251DE33CF281802D0E7001A0958C3B6EB542D2014340010B89112E228803518E2047E0004322B4128352DFE72BFE1CC77000E226B92FF7F7F0F4899CCEB788FBA632A444019349E40A801CA941898B661ECBC40820061A78E254024C126797B31A804B27C0582B2D7D4AF02791E431531100B2458A6219D29CB6C4247F7D6DB27BCBA4065138014C05B00801CC0513280108047020106460079801000332200B60002832801C200718012801503801A800B02801723F9B90009D6600D44A87B0CC8010B89D0661F980331F20A44470076767F8EF75AA94F5E1E6E9790C9008BF801AB8002171CA2A45C100661FC508B911C8043EC00C224BB8A753A6677FDB7B8EA85932F4600BE0039138612F684AB86392889C4A201253C013100623D464834200CC1787D09E76FC78200A16603A543E6D9E695E4C74C012D004646D08CAF74391B4232BDD1E4FFEE033805B3DAB074ACF351399FCCEA5F592697E1CB802B2D1D0BCFE410C015B004E46BE17973C949C213153005A6932C0129BDF675DD2CBF3482401BE7802D37AA4DFE6F549BD4A42363A200D5F40149985FEDF2ACF35AB4BD3003004A730F74019B8803F08A0943B1007A21C2487C0002DC578BC600A497B35A8050020F24432444401415002AF07A7F7FE004DB93001A931FC33A802B37FB517A4A52254010E2374C637895BF7E5CC66F53EB0CC2F4C92080292B1E7A0DB26BE6008CE1ACC801804938F530A1227F2A6A4004349A31009F7801A900021908A18C5D100722C43C8F9312CFD4040269934949661E0096FE75092ACA4F0B6A005CD6CBE1218027258AA3F00439377F5D566E338D121C0239DD9C4942FA4E8F73DFA62656402704E523896FAE9E00B4E779DE6BF15595C56DBF0ACD391802F400FA4FEADD769FD5BAE7318FCF32AB8"""

In [77]:
len(task)

1404

In [73]:
len(task)*4

5616

In [28]:
from icecream import ic

In [34]:
# packet = "D2FE28"
packet = "38006F45291200"

s = f"{int(packet, base=16):b}"
ic(s)

v, t, ns = s[:3], s[3:6], s[6:]
if t == '100':
    ic('Its a number')
    numbers = [int(s[7+c:6+c+5], base=2) for c in range(0, len(s[6:]), 5) if len(s[6+c:6+c+5]) == 5]
    number = sum([numbers[i] << (4*(len(numbers) - 1 - i)) for i in range(len(numbers))])
    result = (v, t, number)
else:
    ic('Its an operator')
    result = (v, t)
ic(result)

ic| s: '111000000000000110111101000101001010010001001000000000'
ic| 'Its an operator'
ic| result: ('111', '000')


('111', '000')

In [74]:
def parse(packet):
    for s in packet:
        for c in f"{int(s, base=16):04b}":
            yield c

cnt = 0
def take(gen, n):
    global cnt
    cnt += 1
    return [next(gen) for _ in range(n)]

def read_as_number(gen, n):
    return int("".join(take(gen, n)), base=2)

In [75]:
# VERSION = 0

# packet = "D2FE28"
# packet = "38006F45291200"
# packet = "620080001611562C8802118E34"
packet = task
# packet = "C200B40A82"

gen = parse(packet)

def extract(gen) -> Tuple[int, int, Any]:
    # ic("====")
    v = read_as_number(gen, 3)
    # VERSION += v
    t = read_as_number(gen, 3)
    # ic(v, t)
    bits_read = 6
    if t != 4:
        # ic("Operator")
        I = read_as_number(gen, 1)
        bits_read += 1
        length = read_as_number(gen, 15 if I == 0 else 11)
        # ic(length)

        args = []

        if I == 0:
            read = 0
            while read < length:
                result = extract(gen)
                read += result[0]
                args.append(result[1:])
        else:
            for _ in range(length):
                result = extract(gen)
                args.append(result[1:])                

        return (v, t, args)
    else:
        # ic("Literal")
        proceed = True
        number = 0
        while proceed:
            proceed = bool(read_as_number(gen, 1))
            bits_read += 1
            number = number << 4
            number += read_as_number(gen, 4)
            bits_read += 4
        return bits_read, v, t, number

root = extract(gen)
# VERSION

StopIteration: 

In [76]:
cnt

1787